In [ ]:
# !pip install pandas-summary ipython fastai==0.7
!pip install treeinterpreter

In [ ]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
import IPython

from sklearn import metrics

In [ ]:
PATH = '../input/train/'
# Kaggle Kernel
# !mkdir data && cp -r ../input/train data/bulldozer
# !ls data/bulldozer -a

In [ ]:
df_raw = pd.read_csv(f'{PATH}Train.csv', low_memory=False, parse_dates=["saledate"])
# ! ls ../input/train
# df_raw = pd.read_feather('tmp/bulldozers-raw')

In [ ]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 1000): 
            display(df)
display_all(df_raw.tail().transpose())

In [ ]:
df_raw.SalePrice = np.log(df_raw.SalePrice)

In [ ]:
# df_raw.saledate.head
add_datepart(df_raw, 'saledate')

In [ ]:
# df_raw.saleYear.head()
# display_all(df_raw.tail().transpose())
train_cats(df_raw)
df_raw.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)
df_raw.UsageBand = df_raw.UsageBand.cat.codes

In [ ]:
display_all(df_raw.isnull().sum().sort_index()/len(df_raw))

In [ ]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/bulldozers-raw')

In [ ]:
x, y, nas = proc_df(df_raw, 'SalePrice')

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(x, y)
m.score(x, y)

In [ ]:
def split_vals(a, n): return a[:n].copy(), a[n:].copy()
n_valid = 12000
n_trn = len(x) - n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(x, n_trn)
y_train, y_valid = split_vals(y, n_trn)
X_train.shape, y_train.shape, X_valid.shape

In [ ]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())
def print_score(m):
    res = [rmse(m.predict(X_train), y_train),
           rmse(m.predict(X_valid), y_valid),
           m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(x, y)
m.score(x, y)
print_score(m)

In [ ]:
df_raw.SalePrice = np.log(df_raw.SalePrice)

In [ ]:
df_trn, y_trn, nas = proc_df(df_raw, 'SalePrice', subset=30000, na_dict=nas)
X_train, _ = split_vals(df_trn, 20000)
y_train, _ = split_vals(y_trn, 20000)

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

In [ ]:
m = RandomForestRegressor(n_estimators=1, max_depth=3, bootstrap=False, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
draw_tree(m.estimators_[0], df_trn, precision=3)

In [ ]:
?IPython

In [ ]:
m = RandomForestRegressor(n_estimators=1, bootstrap=False, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
preds = np.stack([t.predict(X_valid) for t in m.estimators_])
preds[:,0], np.mean(preds[:,0]), y_valid[0]

In [ ]:
m = RandomForestRegressor(n_estimators=40, n_jobs=-1, 
                          oob_score=True)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
# df_trn, y_trn = proc_df(df_raw, 'SalePrice')
# X_train, X_valid = split_vals(df_trn, n_trn)
# y_train, y_valid = split_vals(y_trn, n_trn)
# set_rf_samples(20000)

In [ ]:
fi = rf_feat_importance(m, X_train); fi[:10]

In [ ]:
fi.plot('cols', 'imp', figsize=(10,6), legend=False);

In [ ]:
 def plot_fi(fi):
  return fi.plot('cols','imp','barh', figsize=(12,7), legend=False)
plot_fi(fi[:30]);


In [ ]:
set_rf_samples(50000)

m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, 
        max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
from treeinterpreter import treeinterpreter as tl

In [ ]:

df_train, df_valid = split_vals(df_raw[df_trn.columns], n_trn)

In [1]:
row = X_valid.values[None,0]; row

NameError: name 'X_valid' is not defined

In [2]:
prediction, bias, contributions = ti.predict(m, row)

NameError: name 'ti' is not defined

In [ ]:
contributions[0].sum()